In [1]:
import midi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import keras
import warnings
import pickle
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error
warnings.filterwarnings("ignore")
%matplotlib inline

Using TensorFlow backend.


In [27]:
class MidiTokenizer():
    def __init__(self, filepath, padding=True):
        '''
        filepath: path to MIDI file
        padding: whether or not to pad time series data in case of polyphony
        self.mid: python-midi parsing of MIDI file
        self.format: format of MIDI file (0, 1, 2)
        self.tick_relative: whether event ticks are recorded relative to other
            events or absolutely
        '''
        self.mid = midi.read_midifile(filepath)
        self.format = self.mid.format
        self.tick_relative = self.mid.tick_relative
        #set ticks relative to true automatically
        self.padding = padding
        
        self.event_data = {i: [] for i in range(17)}
        #initialize event data to empty 
        self.time_series_data = {}
        #init time series to empty
        
        if self.format == 0:         # one midi track
            self.parse_track([mid], self.tick_relative)
        elif self.format == 1:       # multiple midi tracks
            for track in self.mid:
                self.parse_track(track, self.tick_relative)
                
        for i in self.event_data.keys():
            self.event_data[i] = pd.DataFrame(
                self.event_data[i], columns=['start', 'duration', 'pitch', 'velocity'])
            
        self.gen_time_series_data()
                            
    def parse_track(self, track, tick_relative=True):        
        tick = 0
        pitch_data = {}
        total_counter = 0
        event_tick = 0
        non_tick = 0
        for event in track:
            if type(event) != midi.NoteOnEvent and type(event) != midi.NoteOffEvent:
                # if event does not correspond to a note, skip event
                continue

            tick = (tick + event.tick) if tick_relative else event.tick
            
            
            if event.velocity != 0 and type(event) != midi.NoteOffEvent:
                # note on
                datum = [tick, None, event.pitch, event.velocity]
                self.event_data[event.channel].append(datum)
                try:
                    if pitch_data[event.pitch][-1][1] == None:
                        pitch_data[event.pitch][-1][1] = tick
                        
                    pitch_data[event.pitch].append(datum)
                except KeyError:
                    pitch_data[event.pitch] = [datum]
            else:
                # note off
                pitch_data[event.pitch][-1][1] = tick - self.event_data[event.channel][-1][0]
            #print(pitch_data)                 
    def gen_time_series_data(self):
        #count_track = 0
        
        for channel, parsed_track in self.event_data.items():
            #for each parsed_track within the event data parse thru
            
            #print(channel)
            #print("_______________")
            if parsed_track.shape[0] == 0:
                continue
            
            ticks = parsed_track[['start', 'duration']].values
            #attain start and duration values 
            max_tick = max(map(lambda x:np.sum(x), ticks)) + 1
            #print(max_tick)
            self.time_series_data[channel] = [[] for _ in range(max_tick)]
            #get max tick (total size for the channel) and initialize to empty to then create events
            
            #print(self.times_series_data)
            for _, event in parsed_track.iterrows():
                start, duration, pitch, velocity = event.values
                #get the start tick, duration, pitch and value and set to theh event event values
                end = start + duration
                note = (pitch, velocity)
                                
                self.time_series_data[channel][start].append(note)
                self.time_series_data[channel][end].append(note)
                
            activated_notes = set([])
            #print(self.time_series_data)
            for i, notes in enumerate(self.time_series_data[channel]):
                for note in notes:
                    if note not in activated_notes:
                        activated_notes.add(note)
                    else:
                        activated_notes.remove(note)
                
                self.time_series_data[channel][i] = list(activated_notes)
              
            if self.padding:
                self.time_series_data[channel] = \
                    keras.preprocessing.sequence.pad_sequences(self.time_series_data[channel])
        #print(self.event_data.items())

            

In [28]:
mt = MidiTokenizer('data/big_poppa/BigPoppa.mid')

0
_______________________________________________________________________________
     start  duration  pitch  velocity
0        0       357     67        98
1        0       362     76        98
2        0       366     72       119
3        0       359     69       119
4      480       368     77       107
5      480       398     71       107
6      480       383     68        90
7      480       381     74        98
8      960       130     71        86
9      960       137     67        81
10     960       147     76        98
11     960       151     72        90
12    1140        51     76        98
13    1140        46     67        90
14    1140        60     69        98
15    1140        67     72        98
16    1440       120     65        98
17    1440       120     69       107
18    1440       120     74        98
19    1440       120     70       107
20    1680       110     69       119
21    1680       110     65       107
22    1680       110     74       107
23    

_______________________________________________________________________________
    start  duration  pitch  velocity
0    6720         3     71       103
1    6840        58     69        91
2    6900       380     71        91
3    7320         3     72       103
4    7500       146     74       103
5   14400         3     71       103
6   14520        58     69        91
7   14580       380     71        91
8   15000         3     72       103
9   15180       146     74       103
10  22080         3     71       103
11  22200        58     69        91
12  22260       380     71        91
13  22680         3     72       103
14  22860       146     74       103
15  30540       146     74       103
16  37440         3     71       103
17  37560        58     69        91
18  37620       380     71        91
19  38040         3     72       103
20  38220       146     74       103
21  45120         3     71       103
22  45240        58     69        91
23  45300       380     71      

In [4]:
mt.event_data[2]

,start,duration,pitch,velocity
0,0,96,29,121
1,180,64,29,121
2,360,56,26,99
3,480,90,28,99
4,960,90,33,109
5,1320,54,32,99
6,1440,115,31,121
7,1680,105,36,121
8,1920,91,29,121
9,2100,67,29,121


In [13]:
def process_data(data, timestep):
    X, y = [], []
    for i in range(len(data)-timestep-1):
        X.append(np.array([data[i:(i+timestep)]]))
        y.append(np.array([data[(i+timestep)]]))
        
    X, y = np.array(X), np.array(y)
    return X.reshape(*[_ for _ in X.shape if _ != 1]), \
           y.reshape(*[_ for _ in y.shape if _ != 1])

def cantor_pair(k1, k2):
    return int((k1+k2)*(k1+k2+1)/2+k2)

def cantor_unpair(z):
    w = int((np.sqrt(8*z+1)-1)/2)
    t = (w**2+w)/2
    y = int(z-t)
    x = int(w-y)
    return x, y

In [14]:
class RNN:
    def __init__(self, X, train_test_split=0.8, epochs=100, batch_size=10, lstm_units=128, timestep=32):
        self.X = X
        self.split = int(self.X.shape[0]*train_test_split)
        self.X_train, self.X_test = self.X[:self.split], self.X[self.split:]
        self.epochs = epochs
        self.batch_size = batch_size
        self.lstm_units = lstm_units
        self.timestep = timestep
        self.scaler = StandardScaler()
        self.scale_data()
        
        self.model = Sequential()
        self.model.add(
            LSTM(self.lstm_units, input_shape=(
                self.timestep, self.X_train_processed.shape[-1]
            ), activation='relu')
        )
        self.model.add(Dense(self.X_train_processed.shape[-1]))
        self.model.compile(optimizer='adamax', loss='mse', metrics=['accuracy'])
        
    def flatten_data(self):
        self.X_train_flattened = np.apply_along_axis(lambda x:x[0], 2, self.X_train)
        self.X_test_flattened = np.apply_along_axis(lambda x:x[0], 2, self.X_test)
        
    def scale_data(self):
        self.flatten_data()
        self.scaler.fit(self.X_train_flattened)
        self.X_train_scaled = self.scaler.transform(self.X_train_flattened)
        self.X_test_scaled = self.scaler.transform(self.X_test_flattened)
        
        self.X_train_processed, self.y_train_processed = \
            process_data(self.X_train_scaled, self.timestep)
        self.X_test_processed, self.y_test_processed = \
            process_data(self.X_test_scaled, self.timestep)
        
    def train(self, verbose=0):
        history = self.model.fit(self.X_train_processed, self.y_train_processed,
                                 batch_size=self.batch_size, epochs=self.epochs)
        
        return history
    
    def evaluate(self):
        keras.evaluate(x=self.y_test_processed,
                       y=self.model.predict())

In [18]:
len(mt.time_series_data[0])

167921

In [17]:
mt.time_series_data[5]

array([[[ 0,  0],
        [ 0,  0]],

       [[ 0,  0],
        [ 0,  0]],

       [[ 0,  0],
        [ 0,  0]],

       ...,

       [[ 0,  0],
        [74, 57]],

       [[ 0,  0],
        [74, 57]],

       [[ 0,  0],
        [ 0,  0]]], dtype=int32)

In [40]:
rnn = RNN(mt.time_series_data[0], epochs=5)

In [41]:
rnn.train(verbose=1)

Epoch 1/5
134303/134303 [==============================] - 427s 3ms/step - loss: 0.2888 - acc: 0.8207
Epoch 2/5
 30610/134303 [=====>........................] - ETA: 5:29 - loss: 0.0162 - acc: 0.8559

KeyboardInterrupt: 

In [15]:
mean_squared_error(rnn.scaler.inverse_transform(rnn.y_test_processed),
                   rnn.scaler.inverse_transform(
                       rnn.model.predict(rnn.X_test_processed)
                   ))

11.542647420002702

In [16]:
rnn.model.evaluate(rnn.X_test_processed, rnn.y_test_processed)

1596/1596 [==============================] - 0s 269us/step


[0.031311907982897194, 0.9956140350877193]

In [17]:
pred = rnn.scaler.inverse_transform(rnn.model.predict(rnn.X_train_processed))

In [18]:
def note_map(x):
    if x < 0:
        return 0
    elif x > 127:
        return 0
    else:
        return x

In [19]:
note_map = np.vectorize(note_map)
pred = note_map(pred)

In [20]:
pattern = midi.Pattern()
track = midi.Track()
pattern.append(track)
track.append(midi.PortEvent(tick=0, data=[0]))
track.append(midi.TrackNameEvent(tick=0, text='Thank you Kanye, very cool!'))
track.append(midi.ProgramChangeEvent(tick=0, channel=8))

In [89]:
pattern2 = midi.Pattern()
track2 = midi.Track()

pred = rnn.scaler.inverse_transform(rnn.y_train_processed).astype(int)
prev = []
counter_off = 0
for tick, note_arr in enumerate(pred[:-1]):
    for note in note_arr:
        print()
        if note == 0: 
            continue
        
        if note in prev:
            if note not in pred[tick+1]:
                #print(pred[tick+1])
                track2.append(midi.NoteOnEvent(tick=tick, channel=10, data=[note, 0]))
                counter_off+=1
        else:
            track2.append(midi.NoteOnEvent(tick=tick, channel=10, data=[note, 60]))
            
    prev = note_arr
    
for i, event in reversed(list(enumerate(track2))):
    if i == 0:
        continue
        
    event.tick = (event.tick - track2[i-1].tick)
    
pattern2.append(track2)

[ 0  0  0 76 69 72]
[ 0  0  0  0 76 72]
[ 0  0  0  0  0 72]
[0 0 0 0 0 0]
[ 0  0  0 70 74 68]
[ 0  0  0 70 74 68]
[ 0  0  0  0 71 68]
[ 0  0  0  0 71 68]
[ 0  0  0  0  0 71]
[0 0 0 0 0 0]
[ 0  0  0 76 67 72]
[ 0  0  0  0 76 72]
[ 0  0  0  0  0 72]
[0 0 0 0 0 0]
[ 0  0  0 76 72 69]
[ 0  0  0  0 72 69]
[ 0  0  0  0  0 72]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[ 0  0  0 72 76 67]
[ 0  0  0  0 72 76]
[ 0  0  0  0  0 76]
[0 0 0 0 0 0]
[ 0  0  0 74 71 68]
[ 0  0  0  0 74 71]
[ 0  0  0  0  0 71]
[0 0 0 0 0 0]
[ 0  0  0 67 72 71]
[ 0  0  0  0 72 71]
[ 0  0  0  0  0 72]
[0 0 0 0 0 0]
[ 0  0  0 69 76 72]
[ 0  0  0  0 69 72]
[ 0  0  0  0  0 69]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[ 0  0  0  0 74 69]
[ 0  0  0  0  0 74]
[0 0 0 0 0 0]
[ 0  0  0 72 68 74]
[ 0  0  0  0 72 68]
[ 0  0  0  0  0 72]
[0 0 0 0 0 0]
[ 0  0  0 76 72 

In [88]:
print(counter_off)
print(len(pred))

943
134303


In [22]:
midi.write_midifile("mary_new_og.mid", pattern2)

In [23]:
pred = rnn.scaler.inverse_transform(rnn.model.predict(rnn.X_train_processed)).astype(int)

In [44]:
print(type(pred))

<class 'numpy.ndarray'>


In [24]:
pattern3 = midi.Pattern()
track3 = midi.Track()

pred2 = note_map(rnn.scaler.inverse_transform(rnn.model.predict(rnn.X_train_processed)).astype(int))
prev = []

for tick, note_arr in enumerate(pred2[:-1]):
    for note in note_arr:
        if note == 0: continue
        
        if note in prev:
            if note not in pred[tick+1]:
                track3.append(midi.NoteOnEvent(tick=tick, channel=10, data=[note, 0]))
        else:
            track3.append(midi.NoteOnEvent(tick=tick, channel=10, data=[note, 60]))
            
    prev = note_arr
    
for i, event in reversed(list(enumerate(track3))):
    if i == 0:
        continue
        
    event.tick = (event.tick - track3[i-1].tick)
    
pattern3.append(track3)

In [25]:
midi.write_midifile("mary_new_out.mid", pattern3)

In [26]:
def one_d_convert(datum):
    return datum.reshape(1, *datum.shape)

In [163]:
def babl(model, length, seed):
    X, Y = copy.deepcopy(seed), []
    init = model.predict(one_d_convert(X))
    
    for i in range(length):
        y = rnn.model.predict(one_d_convert(X))
        #print(y[0])
        Y.append(y[0])
        X = np.append(X[-31:], y, axis=0)

    return np.array(Y)

In [170]:
Y = babl(rnn.model, 100000, rnn.X_test_processed[0])

In [165]:
print(Y[0:10])

[[-0.12760966 -0.13388182 -0.6361828  -0.7427971   0.6484285   0.5607809 ]
 [-0.12711036 -0.1340625  -0.63354313 -0.7425012   0.6398401   0.55528516]
 [-0.12697521 -0.13429414 -0.6315686  -0.74130416  0.6316109   0.5497552 ]
 [-0.12698469 -0.13452537 -0.62983626 -0.73947084  0.6231423   0.54390097]
 [-0.12708978 -0.13474213 -0.62828    -0.7370642   0.6144345   0.5377665 ]
 [-0.12725633 -0.13492307 -0.6268534  -0.73415077  0.6055327   0.53140634]
 [-0.12745969 -0.13505094 -0.6255067  -0.73076284  0.5964481   0.5248704 ]
 [-0.12766382 -0.13510847 -0.6242011  -0.726964    0.58718944  0.5181984 ]
 [-0.12784596 -0.13508333 -0.62290305 -0.7227906   0.57775986  0.5114182 ]
 [-0.1279916  -0.13496661 -0.62158483 -0.7182637   0.5681561   0.50454867]]


In [171]:
Y = note_map(rnn.scaler.inverse_transform(Y).astype(int))

In [161]:
print(Y[0:10])

[[ 0  0  0  0 73 71]
 [ 0  0  0  0 72 71]
 [ 0  0  0  0 72 71]
 [ 0  0  0  0 72 71]
 [ 0  0  0  0 72 71]
 [ 0  0  0  0 71 70]
 [ 0  0  0  0 71 70]
 [ 0  0  0  0 71 70]
 [ 0  0  0  0 70 70]
 [ 0  0  0  0 70 70]]


In [60]:
print(pred2)

[[56 64]
 [56 64]
 [56 64]
 ...
 [ 3 61]
 [ 3 61]
 [ 3 61]]


In [132]:
print(Y.shape)

(30000, 6)


In [54]:
print(type(Y))

<class 'numpy.ndarray'>


In [79]:
pattern4 = midi.Pattern()
track4 = midi.Track()

Y = note_map(rnn.scaler.inverse_transform(Y).astype(int))
prev = []

for tick, note_arr in enumerate(Y[:-1]):
    for note in note_arr:
        if note == 0: continue
        
        if note in prev:
            if note not in Y[tick+1]:
                track4.append(midi.NoteOnEvent(tick=tick, channel=10, data=[note, 0]))
        else:
            track4.append(midi.NoteOnEvent(tick=tick, channel=10, data=[note, 60]))
            
    prev = note_arr
    
for i, event in reversed(list(enumerate(track4))):
    if i == 0:
        continue
        
    event.tick = (event.tick - track4[i-1].tick)
    
pattern4.append(track4)

In [80]:
midi.write_midifile("mary_neww_song.mid", pattern4)

In [67]:
with open('pred', 'rb') as f:
    pred = pickle.load(f)

In [68]:
print(track4)

midi.Track(\
  [])


In [133]:
def detokenize_data(pred):
   pattern = midi.Pattern()
   track = midi.Track()
   prev = []

   for tick, note_arr in enumerate(pred[:-1].tolist()):
       for note in note_arr:
           if note == 0:
               continue

           if note in prev:
               if note not in pred[tick+1]:
                   track.append(midi.NoteOffEvent(tick=tick, channel=10, data=[note, 0]))
           else:
               if note in pred[tick+1]:
                   track.append(midi.NoteOnEvent(tick=tick, channel=10, data=[note, 60]))

       prev = note_arr

   for i, event in reversed(list(enumerate(track))):
       if i == 0:
           continue

       event.tick = (event.tick - track[i-1].tick)

   pattern.append(track)

   return pattern

In [172]:
blah = detokenize_data(Y)

In [173]:
print(Y.shape)

(100000, 6)


In [174]:
print(blah)

midi.Pattern(format=1, resolution=220, tracks=\
[midi.Track(\
  [midi.NoteOnEvent(tick=0, channel=10, data=[71, 60]),
   midi.NoteOnEvent(tick=1, channel=10, data=[72, 60]),
   midi.NoteOffEvent(tick=3, channel=10, data=[72, 0]),
   midi.NoteOnEvent(tick=1, channel=10, data=[70, 60]),
   midi.NoteOffEvent(tick=2, channel=10, data=[71, 0]),
   midi.NoteOffEvent(tick=3, channel=10, data=[70, 0]),
   midi.NoteOnEvent(tick=0, channel=10, data=[69, 60]),
   midi.NoteOnEvent(tick=1, channel=10, data=[1, 60]),
   midi.NoteOffEvent(tick=2, channel=10, data=[69, 0]),
   midi.NoteOffEvent(tick=0, channel=10, data=[69, 0]),
   midi.NoteOnEvent(tick=1, channel=10, data=[68, 60]),
   midi.NoteOnEvent(tick=0, channel=10, data=[68, 60]),
   midi.NoteOnEvent(tick=2, channel=10, data=[2, 60]),
   midi.NoteOnEvent(tick=1, channel=10, data=[67, 60]),
   midi.NoteOffEvent(tick=1, channel=10, data=[68, 0]),
   midi.NoteOffEvent(tick=1, channel=10, data=[2, 0]),
   midi.NoteOnEvent(tick=1, channel=10, data=

In [175]:
midi.write_midifile("other_600.mid", blah)